# What are Named Entities?

Named entities are words or phrases that refer to specific persons, organizations, locations, dates, quantities, or other real-world objects or concepts.

In the sentence:

"Apple CEO Tim Cook spoke in New York on Monday"

what are the named entities?

*"Apple" (organization), "Tim Cook" (person), "New York" (location), and "Monday" (time)*

# Techniques for Named Entity Extraction

Can you name some techniques we can use for Named Entity extraction?

Let's build a rule-based Named Entity extraction system. Here's your passage:


> The Ministry of Education announced at 1 PM on 12th of October 2024 that Perera scored the highest in the Advanced Level examination at Royal College. Captain Cook Restaurant near Lake Gardens is owned by Mr. Silva, who used to teach at Trinity. While Elephant House ice cream remains popular, new brands like Happy Cow from Green Farms Ltd. are gaining market share. The Morning News reported that Dialog's CEO will speak at SLIIT next Friday about Smart Living solutions. The company's annual meeting is scheduled for 1300 10/12/2024 at Unity Plaza. Meanwhile, Unity Plaza shops are preparing for the April Season sales, which coincide with both New Year celebrations. Eagle Insurance (now merged with Union Bank) opened their branch opposite Victoria Park, where Peace Pagoda is clearly visible. Speaking of peace, Peace Cola is launching their new drink called Life, competing with Sprite and Sprint.

Now write a function that extracts all
-- persons
-- organisations
-- locations
-- brands
-- dates or time



# Named Entity Extraction with BERT

## What is BERT?

Read the paper: https://arxiv.org/pdf/1810.04805

Read this blog to understand transformers: https://jalammar.github.io/illustrated-transformer/

## CoNLL dataset

Read more about it here: https://www.clips.uantwerpen.be/conll2003/ner/

In [ ]:
%pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [ ]:
print(dataset["train"].features)

{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None), 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}


In [ ]:
# Let's inspect the dataset
sentence_0 = dataset["train"][0]

Can you identify each of the keys?

Explain:
- id
- tokens
- pos_tags
- chunk_tags
- ner_tags



In [ ]:
sentence_0_str = " ".join(dataset["train"][0]['tokens'])
sentence_0_str

'EU rejects German call to boycott British lamb .'

In [ ]:
# map pos tag numbers to their labels
pos_tags = dataset["train"].features["pos_tags"].feature.names
chunk_tags = dataset["train"].features["chunk_tags"].feature.names
ner_tags = dataset["train"].features["ner_tags"].feature.names

print(pos_tags)
print(chunk_tags)
print(ner_tags)


['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP']
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [ ]:
print("POS Tags for sentence_0")
for i, pos_ in enumerate(sentence_0['pos_tags']):
    print(f"{pos_}  \t- {pos_tags[pos_]} \t- {sentence_0['tokens'][i]}")

POS Tags for sentence_0
22  	- NNP 	- EU
42  	- VBZ 	- rejects
16  	- JJ 	- German
21  	- NN 	- call
35  	- TO 	- to
37  	- VB 	- boycott
16  	- JJ 	- British
21  	- NN 	- lamb
7  	- . 	- .


In [ ]:
print("Chunk Tags for sentence_0")
for i, chunk_ in enumerate(sentence_0['chunk_tags']):
    print(f"{chunk_} \t- {chunk_tags[chunk_]} \t- {sentence_0['tokens'][i]}")

Chunk Tags for sentence_0
11 	- B-NP 	- EU
21 	- B-VP 	- rejects
11 	- B-NP 	- German
12 	- I-NP 	- call
21 	- B-VP 	- to
22 	- I-VP 	- boycott
11 	- B-NP 	- British
12 	- I-NP 	- lamb
0 	- O 	- .


In [ ]:
print("NER Tags for sentence_0")
for i, ner_ in enumerate(sentence_0['ner_tags']):
    print(f"{ner_} \t- {ner_tags[ner_]} \t\t- {sentence_0['tokens'][i]}")

NER Tags for sentence_0
3 	- B-ORG 		- EU
0 	- O 		- rejects
7 	- B-MISC 		- German
0 	- O 		- call
0 	- O 		- to
0 	- O 		- boycott
7 	- B-MISC 		- British
0 	- O 		- lamb
0 	- O 		- .


In [ ]:
# Check the size of the dataset

print(f"Train size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['validation'])}")
print(f"Test size: {len(dataset['test'])}")

Train size: 14041
Validation size: 3250
Test size: 3453


## Dataset preparation for traning

In [ ]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, Dataset

class NERDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.tokenized_dataset = tokenized_dataset

    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        item = self.tokenized_dataset[idx]
        return {
            'input_ids': torch.tensor(item['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(item['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(item['labels'], dtype=torch.long)
        }

def prepare_data():
    # Load the CoNLL-2003 dataset
    dataset = load_dataset("conll2003")

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

    # Get label list from dataset
    label_list = dataset["train"].features["ner_tags"].feature.names

    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"],
            truncation=True,
            is_split_into_words=True,
            padding='max_length',
            max_length=128,
            return_tensors="pt"  # Return PyTorch tensors
        )

        labels = []
        for i, label in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            labels.append(label_ids)

        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    # Tokenize datasets
    tokenized_datasets = dataset.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=dataset["train"].column_names
    )

    return tokenized_datasets, len(label_list)


## Implement the model

In [ ]:
from transformers import BertForTokenClassification
import torch.nn as nn

class NERModel(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertForTokenClassification.from_pretrained(
            'bert-base-cased',
            num_labels=num_labels
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return outputs

## Implement the traning loop

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
#from ipython-input-0-20a04ce0e163 import prepare_data, NERDataset, NERModel # Import necessary functions

# Assuming tokenized_datasets is already preprocessed and tokenized
batch_size = 16

# Call the prepare_data function to get the tokenized datasets
tokenized_datasets, num_labels = prepare_data() # This line is added to call prepare_data

# Create DataLoader for training, validation, and testing
train_loader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size)
test_loader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

# Initialize the model
# num_labels = len(dataset["train"].features["ner_tags"].feature.names) # This line is removed as num_labels is now returned by prepare_data
model = NERModel(num_labels=num_labels)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)

# Total number of training steps = number of batches * number of epochs
num_epochs = 3
total_steps = len(train_loader) * num_epochs  # Calculate total training steps

# Set up the scheduler to adjust the learning rate
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # No warm-up steps
    num_training_steps=total_steps  # Total training steps
)

# Now, call your training function to start training
# train_model(model, train_loader, valid_loader, optimizer, scheduler, device, num_epochs=num_epochs) # Assuming train_model is defined elsewhere

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch
from tqdm import tqdm  # For progress bar during training

# Define a function to calculate accuracy for evaluation
def compute_accuracy(preds, labels):
    preds = torch.argmax(preds, dim=-1)
    correct_preds = (preds == labels).float()
    return correct_preds.sum() / labels.numel()

# Implement the train_model function
def train_model(model, train_dataloader, eval_dataloader, optimizer, scheduler, device, num_epochs=3):
    for epoch in range(num_epochs):
        # Set model to training mode
        model.train()
        total_train_loss = 0
        total_train_acc = 0

        # Training loop
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            # Move the batch to the device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Zero out the gradients for this batch
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            # Backward pass
            loss.backward()

            # Update parameters and learning rate
            optimizer.step()
            scheduler.step()

            # Track total training loss
            total_train_loss += loss.item()

            # Calculate training accuracy for the batch
            acc = compute_accuracy(logits, labels)
            total_train_acc += acc.item()

        # Compute average training loss and accuracy
        avg_train_loss = total_train_loss / len(train_dataloader)
        avg_train_acc = total_train_acc / len(train_dataloader)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Train Accuracy: {avg_train_acc:.4f}")

        # Evaluation after each epoch (optional but recommended)
        model.eval()  # Set the model to evaluation mode
        total_eval_loss = 0
        total_eval_acc = 0

        # Turn off gradients for validation (faster and saves memory)
        with torch.no_grad():
            for batch in tqdm(eval_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Evaluating"):
                # Move the batch to the device
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                # Forward pass (no gradient calculation)
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                # Track total evaluation loss
                total_eval_loss += loss.item()

                # Calculate evaluation accuracy for the batch
                acc = compute_accuracy(logits, labels)
                total_eval_acc += acc.item()

        # Compute average evaluation loss and accuracy
        avg_eval_loss = total_eval_loss / len(eval_dataloader)
        avg_eval_acc = total_eval_acc / len(eval_dataloader)

        print(f"Epoch {epoch+1} | Eval Loss: {avg_eval_loss:.4f} | Eval Accuracy: {avg_eval_acc:.4f}")

    print("Training complete.")


In [ ]:
model_save_path = "ner_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to ner_model.pth


In [ ]:
## your code here
def train_model(model, train_dataloader, eval_dataloader, optimizer, scheduler, device, num_epochs=3):
    model.train()
    # your code here


## Implement the evaluation function

In [ ]:
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=cdb4096cc9413de51924d790cd421838502bceeb289823c11c59d2a8ba26be2c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report
import torch
from tqdm import tqdm  # Optional, for progress visualization

def evaluate_model(model, test_dataloader, id2label, device):
    model.eval()  # Set the model to evaluation mode
    true_labels = []     # To store the actual labels
    predicted_labels = []  # To store the predicted labels

    with torch.no_grad():  # Disable gradient calculations
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            # Move the batch to the device (GPU/CPU)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass to get the logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get the predicted class labels (index with the highest logit value)
            predictions = torch.argmax(logits, dim=-1)

            # Convert predictions and true labels from IDs to label names
            for i, pred in enumerate(predictions):
                true_label_ids = labels[i].cpu().numpy()
                pred_label_ids = pred.cpu().numpy()

                # Only keep labels that are not padding (-100)
                true_label = [id2label[label_id] for label_id in true_label_ids if label_id != -100]
                pred_label = [id2label[label_id] for label_id, true_id in zip(pred_label_ids, true_label_ids) if true_id != -100]

                true_labels.append(true_label)
                predicted_labels.append(pred_label)

    # Print classification report
    print(classification_report(true_labels, predicted_labels))



## Implement the main() function

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load and prepare dataset
    print("Loading and preparing dataset...")
    tokenized_datasets, num_labels = prepare_data()

    # Convert to custom Dataset objects
    train_dataset = NERDataset(tokenized_datasets["train"])
    eval_dataset = NERDataset(tokenized_datasets["validation"])
    test_dataset = NERDataset(tokenized_datasets["test"])

    # Create data loaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=32,  # Increased batch size
        shuffle=True
    )
    eval_dataloader = DataLoader(
        eval_dataset,
        batch_size=32
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=32
    )

    # Model initialization
    model = NERModel(num_labels=num_labels)
    model.to(device)

    # Define optimizer with better parameters
    optimizer = AdamW(
        model.parameters(),
        lr=5e-5,  # Increased learning rate
        weight_decay=0.01,
        eps=1e-8
    )

    # Learning rate scheduler
    num_epochs = 3
    num_training_steps = num_epochs * len(train_dataloader)
    num_warmup_steps = num_training_steps // 10
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    # Training
    print("Starting training...")
    train_model(model, train_dataloader, eval_dataloader, optimizer, scheduler, device, num_epochs=num_epochs)

    # Evaluation
    print("\nEvaluating on validation set...")
    id2label = {
        0: "O",
        1: "B-PER",
        2: "I-PER",
        3: "B-ORG",
        4: "I-ORG",
        5: "B-LOC",
        6: "I-LOC",
        7: "B-MISC",
        8: "I-MISC"
    }

    evaluate_model(model, eval_dataloader, id2label, device)

    return model

In [ ]:
model = main()

Using device: cuda
Loading and preparing dataset...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...


...

Epoch 3 | Train Loss: 0.0127 | Train Accuracy: 0.1129


Epoch 3/3 - Evaluating: 100%|██████████| 102/102 [00:22<00:00,  4.54it/s]


Epoch 3 | Eval Loss: 0.0332 | Eval Accuracy: 0.1224
Training complete.

Evaluating on validation set...


Evaluating: 100%|██████████| 102/102 [00:22<00:00,  4.51it/s]


              precision    recall  f1-score   support

         LOC       0.97      0.96      0.96      1837
        MISC       0.88      0.92      0.90       922
         ORG       0.92      0.93      0.92      1341
         PER       0.96      0.98      0.97      1836

   micro avg       0.94      0.95      0.95      5936
   macro avg       0.93      0.94      0.94      5936
weighted avg       0.94      0.95      0.95      5936



## Implement inference function


In [ ]:
from transformers import AutoTokenizer
import torch
import numpy as np

def prepare_inference(model_path=None):
    """Initialize tokenizer and load model for inference"""
    tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

    # Load trained model if path provided, otherwise use existing model
    if model_path:
        model = torch.load(model_path)

    id2label = {
        0: "O",
        1: "B-PER",
        2: "I-PER",
        3: "B-ORG",
        4: "I-ORG",
        5: "B-LOC",
        6: "I-LOC",
        7: "B-MISC",
        8: "I-MISC"
    }

    return tokenizer, id2label

def inference(text, model, tokenizer, id2label, device='cuda'):
    """
    Perform NER inference on input text

    Args:
        text (str): Input text to analyze
        model: Trained NER model
        tokenizer: BERT tokenizer
        id2label (dict): Mapping from label ids to label names
        device (str): Device to run inference on ('cuda' or 'cpu')

    Returns:
        list: List of tuples containing (word, entity_label)
    """
    ## fill in your code

    # Ensure model is in evaluation mode
    # Tokenize the text

    # Perform inference
    # Convert predictions to labels
    # Align predictions with words
    # Combine words with their predicted labels
    #return labeled_words
    def inference(text, model, tokenizer, id2label, device='cuda'):
    """
    Perform NER inference on input text

    Args:
        text (str): Input text to analyze
        model: Trained NER model
        tokenizer: BERT tokenizer
        id2label (dict): Mapping from label ids to label names
        device (str): Device to run inference on ('cuda' or 'cpu')

    Returns:
        list: List of tuples containing (word, entity_label)
    """

    model.eval()

    encoding = tokenizer.encode_plus(
        text.split(),
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        return_attention_mask=True
    )


    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1).cpu().numpy()[0]

    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().cpu().numpy())
    word_ids = encoding.word_ids(batch_index=0)  # Get word ID mappings to align subwords

    current_word_id = None
    labeled_words = []
    for idx, label_id in enumerate(predictions):
        word_id = word_ids[idx]

        if word_id is None:
            continue

        if word_id != current_word_id:
            current_word_id = word_id
            word = tokens[idx]
            entity_label = id2label[label_id]
        else:
            word = tokens[idx]
            entity_label = id2label[label_id]
            labeled_words.append((word, entity_label))


    return labeled_words


def print_entities(labeled_words):
    """Pretty print the labeled entities"""
    current_entity = None
    entity_text = []

    for word, label in labeled_words:
        if label == "O":
            if current_entity:
                print(f"{current_entity}: {' '.join(entity_text)}")
                current_entity = None
                entity_text = []
        elif label.startswith("B-"):
            if current_entity:
                print(f"{current_entity}: {' '.join(entity_text)}")
            current_entity = label[2:]
            entity_text = [word]
        elif label.startswith("I-"):
            if current_entity == label[2:]:
                entity_text.append(word)
            else:
                if current_entity:
                    print(f"{current_entity}: {' '.join(entity_text)}")
                current_entity = label[2:]
                entity_text = [word]

    if current_entity:
        print(f"{current_entity}: {' '.join(entity_text)}")

In [ ]:
# First initialize
tokenizer, id2label = prepare_inference()

# Example texts to analyze
texts = [
    "John Smith works at Microsoft in Seattle and visited New York last summer.",
    "The European Union signed a trade deal with Japan in Brussels.",
    "Tesla CEO Elon Musk announced new features coming to their vehicles."
]

# Process each text
for text in texts:
    print("\nText:", text)
    print("Entities found:")
    results = inference(text, model, tokenizer, id2label)
    print_entities(results)


Text: John Smith works at Microsoft in Seattle and visited New York last summer.
Entities found:
PER: John Smith
ORG: Microsoft
LOC: Seattle
LOC: New York

Text: The European Union signed a trade deal with Japan in Brussels.
Entities found:
ORG: European Union
LOC: Japan
LOC: Brussels.

Text: Tesla CEO Elon Musk announced new features coming to their vehicles.
Entities found:
ORG: Tesla
PER: Elon Musk


In [ ]:
# save the model to drive

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/BERT_NER/bert_ner_model.pth"
torch.save(model, save_path)

In [ ]:
model = torch.load(save_path)

<ipython-input-28-f6cbf9c9768c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(save_path)
